In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from tqdm import tqdm

In [2]:
import pandas  as pd
import xgboost as xgb

In [3]:
train_data = pd.read_csv('train.csv')
test_data  = pd.read_csv('test.csv')

In [4]:
train_data.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [5]:
test_data.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0,0.353600,0.738780,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.550540,0.699134,0.286864,0.364515
1,2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.798580,0.446475,0.449037,0.916964,0.513002
2,6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941
3,7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166
4,10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275


In [6]:
train_data['contG'] = train_data['cont1'] * train_data['cont7']
test_data['contG'] = test_data['cont1'] * test_data['cont7']

In [7]:
test_data.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,contG
0,0,0.353600,0.738780,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.550540,0.699134,0.286864,0.364515,0.219481
1,2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.798580,0.446475,0.449037,0.916964,0.513002,0.354135
2,6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941,0.069346
3,7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166,0.182092
4,10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275,0.251386


In [8]:
train_data['contS'] = train_data['cont2'] * train_data['cont9']
test_data['contS'] = test_data['cont2'] * test_data['cont9']

train_data['contR'] = train_data['cont3'] * train_data['cont11']
test_data['contR'] = test_data['cont3'] * test_data['cont11']

In [9]:
train_data['contP'] = train_data['cont4'] * train_data['cont13']
test_data['contP'] = test_data['cont4'] * test_data['cont13']

# train_data['contS'] = train_data['cont5'] * train_data['cont10']
# test_data['contS'] = test_data['cont5'] * test_data['cont10']

In [10]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14','contG', 'contS', 'contR', 'contP']

In [11]:
X_train = train_data[features]
y_train = train_data["target"]
final_X_test = test_data[features]

In [12]:
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (xgb’s “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 

# To use early_stopping_rounds: 
# "Validation metric needs to improve at least once in every 
# early_stopping_rounds round(s) to continue training."

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

# params for XGB are taked from this great kernel https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna 
# by Hamza Ghanmi

In [14]:
regressor = xgb.XGBRegressor(
                 colsample_bytree=0.5,
                 alpha=0.01563,
                 #gamma=0.0,
                 learning_rate=0.01,
                 max_depth=15,
                 min_child_weight=257,
                 n_estimators=4000,                                                                  
                 #reg_alpha=0.9,
                 reg_lambda=0.003,
                 subsample=0.7,
                 random_state=2020,
                 metric_period=100,
                 silent=1)

regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=1)

[0]	validation_0-rmse:7.37014
Will train until validation_0-rmse hasn't improved in 6 rounds.
[1]	validation_0-rmse:7.29717
[2]	validation_0-rmse:7.22493
[3]	validation_0-rmse:7.15341
[4]	validation_0-rmse:7.08263
[5]	validation_0-rmse:7.01255
[6]	validation_0-rmse:6.94321
[7]	validation_0-rmse:6.87455
[8]	validation_0-rmse:6.80660
[9]	validation_0-rmse:6.73930
[10]	validation_0-rmse:6.67272
[11]	validation_0-rmse:6.60678
[12]	validation_0-rmse:6.54151
[13]	validation_0-rmse:6.47690
[14]	validation_0-rmse:6.41296
[15]	validation_0-rmse:6.34965
[16]	validation_0-rmse:6.28701
[17]	validation_0-rmse:6.22497
[18]	validation_0-rmse:6.16356
[19]	validation_0-rmse:6.10279
[20]	validation_0-rmse:6.04261
[21]	validation_0-rmse:5.98307
[22]	validation_0-rmse:5.92411
[23]	validation_0-rmse:5.86576
[24]	validation_0-rmse:5.80800
[25]	validation_0-rmse:5.75084
[26]	validation_0-rmse:5.69425
[27]	validation_0-rmse:5.63824
[28]	validation_0-rmse:5.58281
[29]	validation_0-rmse:5.52794
[30]	validation_

[258]	validation_0-rmse:0.89259
[259]	validation_0-rmse:0.88918
[260]	validation_0-rmse:0.88583
[261]	validation_0-rmse:0.88254
[262]	validation_0-rmse:0.87931
[263]	validation_0-rmse:0.87613
[264]	validation_0-rmse:0.87299
[265]	validation_0-rmse:0.86991
[266]	validation_0-rmse:0.86688
[267]	validation_0-rmse:0.86388
[268]	validation_0-rmse:0.86094
[269]	validation_0-rmse:0.85806
[270]	validation_0-rmse:0.85522
[271]	validation_0-rmse:0.85242
[272]	validation_0-rmse:0.84967
[273]	validation_0-rmse:0.84696
[274]	validation_0-rmse:0.84430
[275]	validation_0-rmse:0.84169
[276]	validation_0-rmse:0.83913
[277]	validation_0-rmse:0.83660
[278]	validation_0-rmse:0.83411
[279]	validation_0-rmse:0.83166
[280]	validation_0-rmse:0.82925
[281]	validation_0-rmse:0.82688
[282]	validation_0-rmse:0.82457
[283]	validation_0-rmse:0.82228
[284]	validation_0-rmse:0.82003
[285]	validation_0-rmse:0.81782
[286]	validation_0-rmse:0.81565
[287]	validation_0-rmse:0.81352
[288]	validation_0-rmse:0.81144
[289]	va

[515]	validation_0-rmse:0.69882
[516]	validation_0-rmse:0.69878
[517]	validation_0-rmse:0.69874
[518]	validation_0-rmse:0.69871
[519]	validation_0-rmse:0.69868
[520]	validation_0-rmse:0.69865
[521]	validation_0-rmse:0.69862
[522]	validation_0-rmse:0.69859
[523]	validation_0-rmse:0.69856
[524]	validation_0-rmse:0.69852
[525]	validation_0-rmse:0.69849
[526]	validation_0-rmse:0.69847
[527]	validation_0-rmse:0.69844
[528]	validation_0-rmse:0.69840
[529]	validation_0-rmse:0.69837
[530]	validation_0-rmse:0.69834
[531]	validation_0-rmse:0.69831
[532]	validation_0-rmse:0.69829
[533]	validation_0-rmse:0.69826
[534]	validation_0-rmse:0.69824
[535]	validation_0-rmse:0.69821
[536]	validation_0-rmse:0.69818
[537]	validation_0-rmse:0.69816
[538]	validation_0-rmse:0.69812
[539]	validation_0-rmse:0.69810
[540]	validation_0-rmse:0.69808
[541]	validation_0-rmse:0.69805
[542]	validation_0-rmse:0.69803
[543]	validation_0-rmse:0.69800
[544]	validation_0-rmse:0.69798
[545]	validation_0-rmse:0.69795
[546]	va

[772]	validation_0-rmse:0.69618
[773]	validation_0-rmse:0.69618
[774]	validation_0-rmse:0.69617
[775]	validation_0-rmse:0.69616
[776]	validation_0-rmse:0.69616
[777]	validation_0-rmse:0.69616
[778]	validation_0-rmse:0.69615
[779]	validation_0-rmse:0.69615
[780]	validation_0-rmse:0.69615
[781]	validation_0-rmse:0.69615
[782]	validation_0-rmse:0.69614
[783]	validation_0-rmse:0.69614
[784]	validation_0-rmse:0.69614
[785]	validation_0-rmse:0.69614
[786]	validation_0-rmse:0.69615
[787]	validation_0-rmse:0.69614
[788]	validation_0-rmse:0.69613
[789]	validation_0-rmse:0.69613
[790]	validation_0-rmse:0.69612
[791]	validation_0-rmse:0.69612
[792]	validation_0-rmse:0.69612
[793]	validation_0-rmse:0.69611
[794]	validation_0-rmse:0.69611
[795]	validation_0-rmse:0.69610
[796]	validation_0-rmse:0.69610
[797]	validation_0-rmse:0.69610
[798]	validation_0-rmse:0.69610
[799]	validation_0-rmse:0.69609
[800]	validation_0-rmse:0.69609
[801]	validation_0-rmse:0.69610
[802]	validation_0-rmse:0.69610
[803]	va

XGBRegressor(alpha=0.01563, base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             metric_period=100, min_child_weight=257, missing=nan,
             monotone_constraints=None, n_estimators=4000, n_jobs=0,
             num_parallel_tree=1, objective='reg:squarederror',
             random_state=2020, reg_alpha=0.0156299993, reg_lambda=0.003,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method=None,
             validate_parameters=False, verbosity=None)

In [18]:
predictions = regressor.predict(final_X_test)

In [19]:
predictions

array([7.8729753, 7.896246 , 7.96356  , ..., 8.223694 , 8.034978 ,
       7.9258127], dtype=float32)

In [20]:
output = pd.DataFrame({'id': test_data.id, 'target': predictions})
output.to_csv('Kaggle_Playground_Submission_XGB1SS.csv', index=False)